# Random Forest Classification

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

## All CpG

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Cognition
df = pd.read_csv('CognitionCateg_Methylome_APPRDXFilter.csv')
print(df.shape)
df.head(3)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Cognition
(458, 123023)


,Sentrix,PATNO,HYS,Gender,APPRDX,Montreal_Cognitive,cg08012045,cg08300930,cg00134295,cg10537824,...,cg10054014,cg15364874,cg06622006,cg22071811,cg24960706,cg06917935,cg02794319,cg10702286,cg10353174,cg19590050
0,200991620021_R01C01,3001,2,1,1,0,0.962437,0.964442,0.243530,0.826172,...,0.852622,0.880126,0.876185,0.770531,0.830047,0.216268,0.234350,0.681550,0.103758,0.837890
1,200973410167_R04C01,3007,2,1,1,0,0.962158,0.963041,0.226060,0.729373,...,0.900772,0.901686,0.906222,0.804494,0.905520,0.254265,0.290337,0.760554,0.075480,0.833824
2,200973410167_R05C01,3010,2,1,1,0,0.958309,0.964922,0.192857,0.765560,...,0.904735,0.898241,0.914635,0.819989,0.897821,0.229338,0.232169,0.775542,0.119109,0.846339


In [3]:
df1 = df.drop(['Sentrix','PATNO','HYS','Gender','Montreal_Cognitive'], axis=1)
df1.head(2)

,APPRDX,cg08012045,cg08300930,cg00134295,cg10537824,cg04990650,cg03379001,cg02215627,cg06699201,cg18100935,...,cg10054014,cg15364874,cg06622006,cg22071811,cg24960706,cg06917935,cg02794319,cg10702286,cg10353174,cg19590050
0,1,0.962437,0.964442,0.24353,0.826172,0.845330,0.852316,0.871498,0.653140,0.89167,...,0.852622,0.880126,0.876185,0.770531,0.830047,0.216268,0.234350,0.681550,0.103758,0.837890
1,1,0.962158,0.963041,0.22606,0.729373,0.830303,0.764861,0.821899,0.717675,0.88783,...,0.900772,0.901686,0.906222,0.804494,0.905520,0.254265,0.290337,0.760554,0.075480,0.833824


In [4]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.5349999999999999
2
Mean accuracy: 0.515
3
Mean accuracy: 0.49499999999999994
4
Mean accuracy: 0.5349999999999999
5
Mean accuracy: 0.48500000000000004
6
Mean accuracy: 0.49000000000000005
7
Mean accuracy: 0.52
8
Mean accuracy: 0.485
9
Mean accuracy: 0.4800000000000001
10
Mean accuracy: 0.465
11
Mean accuracy: 0.5
12
Mean accuracy: 0.4699999999999999
13
Mean accuracy: 0.495
14
Mean accuracy: 0.525
15
Mean accuracy: 0.5249999999999999
16
Mean accuracy: 0.53
17
Mean accuracy: 0.48
18
Mean accuracy: 0.52
19
Mean accuracy: 0.47000000000000003
20
Mean accuracy: 0.595
21
Mean accuracy: 0.5299999999999999
22
Mean accuracy: 0.5000000000000001
23
Mean accuracy: 0.5
24
Mean accuracy: 0.51
25
Mean accuracy: 0.51
26
Mean accuracy: 0.49499999999999994
27
Mean accuracy: 0.5549999999999999
28
Mean accuracy: 0.475
29
Mean accuracy: 0.51
30
Mean accuracy: 0.5199999999999999
31
Mean accuracy: 0.55
32
Mean accuracy: 0.43499999999999994
33
Mean accuracy: 0.575
34
Mean accuracy: 0.45499999

In [5]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Cognition_AllCpG_100RF.csv", index = False)

0.50885


## RanFor 1000 times. Samples = 50

In [6]:
df2 = df1.drop (['APPRDX'], axis=1)

In [7]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(50, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.545
2
Mean accuracy: 0.475
3
Mean accuracy: 0.595
4
Mean accuracy: 0.4800000000000001
5
Mean accuracy: 0.5850000000000001
6
Mean accuracy: 0.47000000000000003
7
Mean accuracy: 0.49000000000000005
8
Mean accuracy: 0.51
9
Mean accuracy: 0.45500000000000007
10
Mean accuracy: 0.525
11
Mean accuracy: 0.595
12
Mean accuracy: 0.49000000000000005
13
Mean accuracy: 0.525
14
Mean accuracy: 0.43
15
Mean accuracy: 0.495
16
Mean accuracy: 0.49000000000000005
17
Mean accuracy: 0.4699999999999999
18
Mean accuracy: 0.475
19
Mean accuracy: 0.5549999999999999
20
Mean accuracy: 0.47000000000000003
21
Mean accuracy: 0.5350000000000001
22
Mean accuracy: 0.45999999999999996
23
Mean accuracy: 0.5199999999999999
24
Mean accuracy: 0.4600000000000001
25
Mean accuracy: 0.5249999999999999
26
Mean accuracy: 0.5149999999999999
27
Mean accuracy: 0.55
28
Mean accuracy: 0.51
29
Mean accuracy: 0.5700000000000001
30
Mean accuracy: 0.53
31
Mean accuracy: 0.4800000000000001
32
Mean accuracy: 0.515
33
Me

In [8]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Cognition_Rand50CpG_100RF.csv", index = False)

0.5059499999999998


## RanFor 100 times. Samples = 97

In [9]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(97, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.505
2
Mean accuracy: 0.45999999999999996
3
Mean accuracy: 0.615
4
Mean accuracy: 0.53
5
Mean accuracy: 0.5549999999999999
6
Mean accuracy: 0.525
7
Mean accuracy: 0.53
8
Mean accuracy: 0.5650000000000001
9
Mean accuracy: 0.5050000000000001
10
Mean accuracy: 0.52
11
Mean accuracy: 0.5800000000000001
12
Mean accuracy: 0.485
13
Mean accuracy: 0.4800000000000001
14
Mean accuracy: 0.45000000000000007
15
Mean accuracy: 0.43500000000000005
16
Mean accuracy: 0.53
17
Mean accuracy: 0.495
18
Mean accuracy: 0.585
19
Mean accuracy: 0.53
20
Mean accuracy: 0.48500000000000004
21
Mean accuracy: 0.54
22
Mean accuracy: 0.48500000000000004
23
Mean accuracy: 0.4600000000000001
24
Mean accuracy: 0.48500000000000004
25
Mean accuracy: 0.485
26
Mean accuracy: 0.585
27
Mean accuracy: 0.5200000000000001
28
Mean accuracy: 0.505
29
Mean accuracy: 0.535
30
Mean accuracy: 0.48
31
Mean accuracy: 0.53
32
Mean accuracy: 0.47000000000000003
33
Mean accuracy: 0.5599999999999999
34
Mean accuracy: 0.585

In [10]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Cognition_Rand97CpG_100RF.csv", index = False)

0.5117499999999999


## RanFor Top 50 Boruta 100 times. Samples = 50

In [11]:
df = pd.read_csv('Cognition_Boruta_Top50_Data.csv')
print(df.shape)
df.head(2)

(458, 52)


,PATNO,APPRDX,cg16541194,cg16333036,cg10410319,cg12345955,cg21043558,cg07456278,cg19690051,cg26090534,...,cg00377727,cg09253187,cg04061361,cg22534925,cg12220605,cg12044671,cg13976366,cg16006242,cg25911279,cg07931157
0,3001,1,0.576279,0.496829,0.021390,0.015210,0.667449,0.582067,0.104141,0.962848,...,0.923318,0.905378,0.568646,0.914514,0.082447,0.869433,0.031021,0.020243,0.031463,0.066176
1,3007,1,0.471834,0.463492,0.020285,0.013241,0.659961,0.667340,0.045867,0.963592,...,0.925809,0.892914,0.719066,0.930051,0.113486,0.755008,0.019411,0.016203,0.020530,0.069590


In [12]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg16541194,cg16333036,cg10410319,cg12345955,cg21043558,cg07456278,cg19690051,cg26090534,cg00417288,...,cg00377727,cg09253187,cg04061361,cg22534925,cg12220605,cg12044671,cg13976366,cg16006242,cg25911279,cg07931157
0,1,0.576279,0.496829,0.021390,0.015210,0.667449,0.582067,0.104141,0.962848,0.892366,...,0.923318,0.905378,0.568646,0.914514,0.082447,0.869433,0.031021,0.020243,0.031463,0.066176
1,1,0.471834,0.463492,0.020285,0.013241,0.659961,0.667340,0.045867,0.963592,0.934470,...,0.925809,0.892914,0.719066,0.930051,0.113486,0.755008,0.019411,0.016203,0.020530,0.069590


In [13]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.6450000000000001
2
Mean accuracy: 0.655
3
Mean accuracy: 0.6399999999999999
4
Mean accuracy: 0.69
5
Mean accuracy: 0.6649999999999999
6
Mean accuracy: 0.6650000000000001
7
Mean accuracy: 0.635
8
Mean accuracy: 0.655
9
Mean accuracy: 0.6
10
Mean accuracy: 0.635
11
Mean accuracy: 0.7
12
Mean accuracy: 0.6399999999999999
13
Mean accuracy: 0.645
14
Mean accuracy: 0.635
15
Mean accuracy: 0.6749999999999999
16
Mean accuracy: 0.7
17
Mean accuracy: 0.6749999999999999
18
Mean accuracy: 0.7050000000000001
19
Mean accuracy: 0.68
20
Mean accuracy: 0.6950000000000001
21
Mean accuracy: 0.6499999999999999
22
Mean accuracy: 0.675
23
Mean accuracy: 0.68
24
Mean accuracy: 0.605
25
Mean accuracy: 0.6799999999999999
26
Mean accuracy: 0.66
27
Mean accuracy: 0.7100000000000001
28
Mean accuracy: 0.6199999999999999
29
Mean accuracy: 0.675
30
Mean accuracy: 0.7
31
Mean accuracy: 0.6699999999999999
32
Mean accuracy: 0.6249999999999999
33
Mean accuracy: 0.685
34
Mean accuracy: 0.62000000000000

In [14]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Cognition_Boruta50CpG_100RF.csv", index = False)

0.6668500000000002


## RanFor Top 50 NMI 100 times. Samples = 50

In [15]:
df = pd.read_csv('Cognition_NMI_Top50_Data.csv')
print(df.shape)
df.head(2)

(458, 51)


,APPRDX,cg16702987,cg16916264,cg14547519,cg04721711,cg10948748,ch.9.1286602F,cg16448286,cg12298429,cg06538998,...,cg01777678,cg23022057,cg25361663,cg21899921,cg00377727,cg10474158,cg15469205,cg15736994,cg09826790,cg15942569
0,1,0.691866,0.870274,0.900845,0.831384,0.183438,0.041805,0.906014,0.032801,0.013463,...,0.797270,0.888828,0.911043,0.032846,0.923318,0.882391,0.865868,0.523965,0.869270,0.888852
1,1,0.713975,0.927010,0.902779,0.875973,0.114013,0.042104,0.890996,0.042715,0.010118,...,0.877015,0.858068,0.925910,0.046905,0.925809,0.897195,0.830760,0.562597,0.914964,0.884358


In [16]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.645
2
Mean accuracy: 0.7
3
Mean accuracy: 0.6500000000000001
4
Mean accuracy: 0.72
5
Mean accuracy: 0.625
6
Mean accuracy: 0.645
7
Mean accuracy: 0.6599999999999999
8
Mean accuracy: 0.705
9
Mean accuracy: 0.675
10
Mean accuracy: 0.635
11
Mean accuracy: 0.665
12
Mean accuracy: 0.6799999999999999
13
Mean accuracy: 0.63
14
Mean accuracy: 0.715
15
Mean accuracy: 0.66
16
Mean accuracy: 0.6500000000000001
17
Mean accuracy: 0.69
18
Mean accuracy: 0.7
19
Mean accuracy: 0.6849999999999999
20
Mean accuracy: 0.655
21
Mean accuracy: 0.68
22
Mean accuracy: 0.675
23
Mean accuracy: 0.72
24
Mean accuracy: 0.615
25
Mean accuracy: 0.675
26
Mean accuracy: 0.605
27
Mean accuracy: 0.665
28
Mean accuracy: 0.64
29
Mean accuracy: 0.6950000000000001
30
Mean accuracy: 0.7
31
Mean accuracy: 0.72
32
Mean accuracy: 0.6050000000000001
33
Mean accuracy: 0.71
34
Mean accuracy: 0.68
35
Mean accuracy: 0.71
36
Mean accuracy: 0.675
37
Mean accuracy: 0.74
38
Mean accuracy: 0.66
39
Mean accuracy: 0.66999

In [17]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Cognition_NMI50CpG_100RF.csv", index = False)

0.6733499999999999


## RanFor Top 50 Boruta + NMI 100 times. Samples = 97

In [18]:
df = pd.read_csv('Cognition_Top50sMerge_Data.csv')
print(df.shape)
df.head(2)

(458, 99)


,PATNO,APPRDX,cg16541194,cg16333036,cg10410319,cg12345955,cg21043558,cg07456278,cg19690051,cg26090534,...,cg24804436,cg01777678,cg23022057,cg25361663,cg21899921,cg10474158,cg15469205,cg15736994,cg09826790,cg15942569
0,3001,1,0.576279,0.496829,0.021390,0.015210,0.667449,0.582067,0.104141,0.962848,...,0.866382,0.797270,0.888828,0.911043,0.032846,0.882391,0.865868,0.523965,0.869270,0.888852
1,3007,1,0.471834,0.463492,0.020285,0.013241,0.659961,0.667340,0.045867,0.963592,...,0.890765,0.877015,0.858068,0.925910,0.046905,0.897195,0.830760,0.562597,0.914964,0.884358


In [19]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg16541194,cg16333036,cg10410319,cg12345955,cg21043558,cg07456278,cg19690051,cg26090534,cg00417288,...,cg24804436,cg01777678,cg23022057,cg25361663,cg21899921,cg10474158,cg15469205,cg15736994,cg09826790,cg15942569
0,1,0.576279,0.496829,0.021390,0.015210,0.667449,0.582067,0.104141,0.962848,0.892366,...,0.866382,0.797270,0.888828,0.911043,0.032846,0.882391,0.865868,0.523965,0.869270,0.888852
1,1,0.471834,0.463492,0.020285,0.013241,0.659961,0.667340,0.045867,0.963592,0.934470,...,0.890765,0.877015,0.858068,0.925910,0.046905,0.897195,0.830760,0.562597,0.914964,0.884358


In [20]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.675
2
Mean accuracy: 0.69
3
Mean accuracy: 0.715
4
Mean accuracy: 0.705
5
Mean accuracy: 0.71
6
Mean accuracy: 0.6900000000000001
7
Mean accuracy: 0.65
8
Mean accuracy: 0.7100000000000001
9
Mean accuracy: 0.6799999999999999
10
Mean accuracy: 0.675
11
Mean accuracy: 0.675
12
Mean accuracy: 0.655
13
Mean accuracy: 0.655
14
Mean accuracy: 0.675
15
Mean accuracy: 0.6900000000000001
16
Mean accuracy: 0.7100000000000001
17
Mean accuracy: 0.71
18
Mean accuracy: 0.72
19
Mean accuracy: 0.7
20
Mean accuracy: 0.7050000000000001
21
Mean accuracy: 0.655
22
Mean accuracy: 0.7200000000000001
23
Mean accuracy: 0.6950000000000001
24
Mean accuracy: 0.705
25
Mean accuracy: 0.6900000000000002
26
Mean accuracy: 0.665
27
Mean accuracy: 0.73
28
Mean accuracy: 0.6449999999999999
29
Mean accuracy: 0.7250000000000001
30
Mean accuracy: 0.7
31
Mean accuracy: 0.7200000000000001
32
Mean accuracy: 0.605
33
Mean accuracy: 0.73
34
Mean accuracy: 0.705
35
Mean accuracy: 0.7150000000000001
36
Mean acc

In [21]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Cognition_Top50sCpG97_100RF.csv", index = False)

0.6865000000000001
